In [ ]:
import os
import pickle

import data
import gravity
import bayes
import numpy as np

import tuples
import util
import matplotlib as mpl
mpl.use('nbAgg')
import matplotlib.pyplot as plt

# Aktualizacja patentów

In [ ]:
base_dir = '/Users/krzysiek/Library/CloudStorage/OneDrive-Osobisty/KR_dzielone/AI_patenty/dane/15_04_2022'
base_dir

In [ ]:
df = data.merge_files(os.path.join(base_dir, 'APPC','*.csv'))
clean_df = data.make_clean_df(df)
with open('../dane/clean_update.pickle','bw') as f:
    pickle.dump(clean_df,f,protocol=pickle.HIGHEST_PROTOCOL)

loaded = data.load_clean('../dane/clean_update.pickle')
assert (loaded == clean_df).all().all()

# Inventor

In [ ]:
df = data.merge_files(os.path.join(base_dir, 'INVC','*.csv'))
clean_df = data.make_clean_df(df,country_column='Inventor country of residence')
with open('../dane/clean_inv.pickle','bw') as f:
    pickle.dump(clean_df,f,protocol=pickle.HIGHEST_PROTOCOL)

loaded = data.load_clean('../dane/clean_inv.pickle')
assert (loaded == clean_df).all().all()

# Kraje others

In [ ]:
df = data.merge_files(os.path.join(base_dir, 'APPC','*.csv'))

all_ciuntries = set(sum([list(c.split('\n')) for c in df['Applicant country of residence'].to_list()],[]))
all_ciuntries = set([tuples.Country(x) for x in all_ciuntries])
len(all_ciuntries.difference(data.Unia))

In [ ]:

print(all_ciuntries.difference(data.Unia))

In [ ]:
clean_df = data.load_clean('../dane/clean_update.pickle')
fractions = data.fractions_countries(clean_df, with_others=True)
dataset = bayes.Dataset.from_pandas(fractions, gravity.CountryFeaturesType.ALL)

In [ ]:

plt.scatter(dataset.x, (dataset.y))
plt.yscale('function', functions=(np.log1p, np.expm1))
plt.yticks([0, 1, 2, 5, 10, 25, 50, 100, 250, 500, 1000])
plt.show()